In [1]:
print("hello world")

hello world


In [26]:
# !uv pip install python-dotenv

In [25]:
# !uv pip install langchain
# !uv pip install langchain-community
# !uv pip install langchain_groq

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:

from langchain_groq.chat_models import ChatGroq

In [5]:
llm = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key=os.getenv("GROQ_API_KEY"))

In [6]:
llm.invoke("What is your strength?")

AIMessage(content="<think>\n\n</think>\n\nHi! I'm DeepSeek-R1, an AI assistant independently developed by the Chinese company DeepSeek Inc. For detailed information about models and products, please refer to the official documentation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 8, 'total_tokens': 50, 'completion_time': 0.208288243, 'prompt_time': 0.01000892, 'queue_time': 0.195422007, 'total_time': 0.218297163}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--46176d87-edc3-4ca2-a800-f41f1db79332-0', usage_metadata={'input_tokens': 8, 'output_tokens': 42, 'total_tokens': 50})

In [7]:
from langchain.tools import tool

In [8]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [9]:
multiply

StructuredTool(name='multiply', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x10ccc9f80>)

In [10]:
multiply.description, multiply.args

('Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [15]:
multiply.invoke({"a":10, "b":20})

200

In [16]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel

In [17]:
class WeatherInput(BaseModel):
    city: str

In [18]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

When we define a tool this way, we have more control over it.

In the below wether_tool, we are validating the input using pydantic model

In [19]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fethcehs real-time weather data for a city",
    args_schema=WeatherInput,
)

In [20]:
weather_tool.args

{'city': {'title': 'City', 'type': 'string'}}

In [21]:
weather_tool.invoke({"city":"zionsville"})

'The weather in zionsville is sunny.'

## Another way of creating a tool

In [22]:
from typing import ClassVar, Type
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

# This is a mock function to simulate fetching weather data.
# In a real-world scenario, this would call a weather API.
def get_weather(city: str, units: str = "metric") -> str:
    """
    Simulates fetching weather data for a given city.
    
    Args:
        city: The name of the city.
        units: The units for temperature ("metric" or "imperial").
        
    Returns:
        A string with a simulated weather report.
    """
    if city.lower() == "london":
        if units.lower() == "imperial":
            return "The weather in London is 68°F and cloudy."
        else:
            return "The weather in London is 20°C and cloudy."
    elif city.lower() == "new york":
        if units.lower() == "imperial":
            return "The weather in New York is 75°F and sunny."
        else:
            return "The weather in New York is 24°C and sunny."
    else:
        return f"Sorry, I cannot get the weather for {city}."

In [23]:
# Define the Pydantic model for the tool's input arguments.
# This ensures that the input data is validated.
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

# Define the StructuredTool class. 
class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"           
    description: ClassVar[str] = (
        "Fetches weather data for a city"
    )
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        """This is the main function that runs the tool's logic."""
        return get_weather(city, units)

In [24]:
# --- How to run the tool ---

# 1. Create an instance of the tool.
# This makes the class ready to be used.
weather_tool = GetWeatherTool()

# 2. Call the tool using its internal _run method.
# You can pass arguments directly.
result_london_metric = weather_tool._run(city="London")
result_london_imperial = weather_tool._run(city="London", units="imperial")
result_new_york = weather_tool._run(city="New York")

# 3. Print the results to see the output.
print(result_london_metric)
print(result_london_imperial)
print(result_new_york)

The weather in London is 20°C and cloudy.
The weather in London is 68°F and cloudy.
The weather in New York is 24°C and sunny.
